# Output Formatters Tutorial

---

In [ ]:
# Initial Setup:
from glob import glob
import xarray as xr
import logging
import os
from datetime import datetime, timezone
import geoips

LOG = logging.getLogger(__name__)

## Importing the GeoIPS Interfaces

Since we are walking through this tutorial as a Python script, 
we need to import every plugin that we plan to use.

We use the convention:
`{interface_type}.get_plugin({plugin_name})` 
to load the plugins at runtime .

In [ ]:
# import every interface type that we are going to use
from geoips.interfaces import readers, algorithms, interpolators, sectors, output_formatters, colormappers, gridline_annotators, feature_annotators

# Since we are reading abi data in a netCDF format, we will use this reader
abi_reader = readers.get_plugin("abi_netcdf")
# Since we will be reading a single channel, we want to use the single_channel algorithm
single_channel_algorithm = algorithms.get_plugin("single_channel")
# We want to use Nearest Neighbor resampling for this dataset, so we will use the interp_nearest interpolator plugin
interp_nearest = interpolators.get_plugin("interp_nearest")
# We want to restrict our output to just the CONUS sector
conus_sector_plugin = sectors.get_plugin("conus")

Since we load these plugins dynamically, it can take some extra work to check
their call signatures.  
We can look more closely at the reader plugin by calling:
(this only works in ipython or jupyter notebooks)
(There are other ways to get this info...)

In [ ]:
abi_reader?

Here we can see the call signature and docstring from the plugin.  
Hopefully this is enough to help you figure out what arguments to pass.

## Reading the ABI Test Data
  
For this test, we will use the data stored in the `GEOIPS_TESTDATA_DIR` directory.
  
We can access the current GeoIPS environment variables by calling:  
`geoips.filenames.base_paths.PATHS`  
which returns a dictionary with the environment variables.

In [ ]:
GEOIPS_TESTDATA_DIR = geoips.filenames.base_paths.PATHS['GEOIPS_TESTDATA_DIR']
in_fpath = f"{GEOIPS_TESTDATA_DIR}/test_data_abi/data/goes16_20200918_1950/*"
in_fpaths = glob(in_fpath)

In [ ]:
# The conus_sector_plugin contains an area_definition.
# We want to use that area_definition when calling our reader.
conus_sector_area = conus_sector_plugin.area_definition

# First we need to read the abi data for channel 14 Brightness Temperature
# xdict = abi_reader(fnames=in_fpaths, chans=["B14BT"])
xdict = abi_reader(fnames=in_fpaths, area_def=conus_sector_area, chans=["B14BT"])

# Since we passed an area_def for "conus", that is the name of the Xarray Object we want to call.
conus_xobj = xdict["conus"]

# Let's look at what conus_xobj contains within it:
conus_xobj

## Applying the Interpolator
Now we will apply the interpolator. Let's look at the call signature:

In [ ]:
interp_nearest?

In [ ]:
# Now let us apply the Nearest Neighbor interpolation to our data
output_dataset = interp_nearest(conus_sector_area,
                                conus_xobj,
                                None,
                                ["B14BT", "longitude", "latitude"])

# Let's look at our output_dataset
output_dataset

In [ ]:
channel_14_bt = output_dataset["B14BT"].data

# Now let us apply the single channel algorithm to our channel 14 Brightness Temperature data
algorithm_output = single_channel_algorithm([channel_14_bt],
                                            output_data_range=[-90.0, 30.0],
                                            input_units="Kelvin",
                                            output_units="celsius")

output_dataset["Infrared"] = xr.DataArray(algorithm_output)

# Let's look at this dataset now that we have created it
output_dataset

In [ ]:
#--------#
# NetCDF #
#--------#

# We can also put our final data into a NetCDF file
ncdf_output_formatter = output_formatters.get_plugin("netcdf_geoips")

timestamp = datetime.strftime(datetime.now(timezone.utc), "%Y%m%d%H%M%S")
GEOIPS_OUTDIRS = geoips.filenames.base_paths.PATHS['GEOIPS_OUTDIRS']
out_fpath = f"{GEOIPS_OUTDIRS}/abi_infrared_xarray_test_{timestamp}.nc"

success_outputs = ncdf_output_formatter(output_dataset,
                                        ["Infrared"],
                                        [out_fpath])
success_outputs

In [ ]:
# We need a colormapper to tell matplotlib what colors we want to use
ir_colormapper = colormappers.get_plugin("Infrared")

ir_color_dict = ir_colormapper()

In [ ]:
#---------------#
# imagery_clean #
#---------------#

# We can also create clean imagery
img_clean_output_formatter = output_formatters.get_plugin("imagery_clean")

out_fpath = f"{GEOIPS_OUTDIRS}/abi_infrared_clean_test_{timestamp}.png"

success_outputs = img_clean_output_formatter(conus_sector_area,
                                             output_dataset,
                                             "Infrared",
                                             [out_fpath],
                                             mpl_colors_info=ir_color_dict)

success_outputs

![image](/home/coleman/geoips/outdirs/abi_infrared_clean_test_20250718183819.png)

In [ ]:
# We need to tell matplotlib how we want the resulting plot to appear
# using a feature_annotator and a gridline_annotator.
# This time we will use the default versions of these plugins.
local_feature_annotator = feature_annotators.get_plugin("default")

local_gridline_annotator = gridline_annotators.get_plugin("default")

In [ ]:
#-------------------#
# imagery_annotated #
#-------------------#

# Now that we have all the setup completed, we can call our output formatter.
# Let's start by loading the imagery_annotated plugin
img_ann_output_formatter = output_formatters.get_plugin("imagery_annotated")

formatter_result_dict = {}
out_fpath = f"{GEOIPS_OUTDIRS}/abi_infrared_annotated_test_{timestamp}.png"

# And then we call the plugin
success_outputs = img_ann_output_formatter(conus_sector_area,
                                           output_dataset,
                                           "Infrared",
                                           [out_fpath],
                                           mpl_colors_info=ir_color_dict,
                                           feature_annotator=local_feature_annotator,
                                           gridline_annotator=local_gridline_annotator,
                                           output_dict=formatter_result_dict)

# Let's print the path to the image on your disk
success_outputs

![image](/home/coleman/geoips/outdirs/abi_infrared_annotated_test_20250718183819.png)

This is using the scripting approach, is that the correct approach?
There are ways to edit the YAML in jupyter notebooks?